In [18]:
"""
@author: Siphamandla Mandindi
"""

# google_colab_flag = True # Make it False if do not intend to use Google Colab and want to train in local machine!!

# # For training in Google Colab
# if(google_colab_flag):
#   from google.colab import drive
#   drive.mount('/content/drive')
#   !ls
#   import sys
#   root_path = '/content/drive/My Drive/Practice/Stratified-k-fold-cross-validation-Image-classification-keras/' # This is the path to where in google drive the code is stored!
#   sys.path.append(root_path)

# # For local training
# else:
root_path = 'C:/Users/ndira/Documents/University_of_the_Witwatersrand/Computer_Vision/Deep_Learning-master/Wits_Research_Report/Stratified-k-fold-cross-validation-Image-classification-keras-master/DS'

# Add all necessary libraries!

In [2]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000
# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

In [3]:
os.listdir('.')

['.ipynb_checkpoints',
 'DS',
 'README.md',
 'stratified_K_fold_CV-X-Rays.ipynb',
 'stratified_K_fold_CV_Shapes.ipynb']

In [4]:
os.chdir('DS')

In [5]:
os.listdir('.')

['test', 'train', 'train_aug', 'validation']

In [6]:
os.chdir('train_aug')

In [7]:
os.listdir('.')

['B-Pneumonia', 'Covid-19', 'Normal', 'Tuberculosis', 'V-Pneumonia']

In [23]:
datasetFolderName=root_path
MODEL_FILENAME=root_path+"model_cv.h5"
sourceFiles=[]
classLabels=['B-Pneumonia', 'Covid-19', 'Normal', 'Tuberculosis', 'V-Pneumonia']
X=[]
Y=[]

img_rows, img_cols =  255, 255 # input image dimensions
train_path=datasetFolderName+'/train_aug/'
validation_path=datasetFolderName+'/validation/'
test_path=datasetFolderName+'/test/'

In [20]:
def transferBetweenFolders(source, dest, splitRate): 
    global sourceFiles
    sourceFiles=os.listdir(source)
    if(len(sourceFiles)!=0):
        transferFileNumbers=int(len(sourceFiles)*splitRate)
        transferIndex=random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source+str(sourceFiles[eachIndex]), dest+str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")
        
def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(datasetFolderName+'/'+source+'/'+label+'/', 
                               datasetFolderName+'/'+dest+'/'+label+'/', 
                               splitRate)

def my_metrics(y_true, y_pred):
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1Score


# First, check if test folder is empty or not, if not transfer all existing files to train.

In [21]:
transferAllClassBetweenFolders('test', 'train', 1.0)

No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!


# Now, split some part of train data into the test folders.

In [25]:
transferAllClassBetweenFolders('train_aug', 'test', 0.20)

In [26]:
def prepareNameWithLabels(folderName):
    sourceFiles=os.listdir(datasetFolderName+'/train_aug/'+folderName)
    for val in sourceFiles:
        X.append(val)
        for i in range(len(classLabels)):
          if(folderName==classLabels[i]):
              Y.append(i)

In [27]:
# Organize file names and class labels in X and Y variables
for i in range(len(classLabels)):
  prepareNameWithLabels(classLabels[i])

In [28]:
X=np.asarray(X)
Y=np.asarray(Y)

In [30]:
X.shape

(19923,)

In [32]:
Y.shape

(19923,)

In [41]:
# Note that, this model structure is a very basic one. To achieve better performance, you should change the model structure and hyperparameters according to your needs and data.
batch_size = 128
epoch=20
activationFunction='relu'

def getModel():
    import tensorflow.keras as tfK
    from tensorflow.keras.applications import Xception

    input_t = tfK.Input(shape=(255,255,3))
    
    conv_base = Xception(weights='imagenet',
                      include_top=False,
                      input_tensor= input_t)
    conv_base.trainable = True
    
    Coromodel = tfK.models.Sequential()
    Coromodel.add(conv_base)
    Coromodel.add(layers.Flatten())
    Coromodel.add(layers.Dropout(0.5))
    Coromodel.add(layers.Dense(256, activation='relu'))
    Coromodel.add(layers.Dense(len(classLabels), activation='softmax'))
    
    Coromodel.compile(loss = 'categorical_crossentropy',
                      optimizer = tf.optimizers.Adam(learning_rate = 1e-4),
                      metrics = ['accuracy'])

    return Coromodel

model=getModel()

# **Stratified K-Fold Cross validation**

In [ ]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=5, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0
for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('validation', 'train_aug', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        for i in range(len(classLabels)):
          if(Y_val[eachIndex]==i):
              classLabel=classLabels[i]
        #Then, copy the validation images to the validation folder
        shutil.move(datasetFolderName+'/train_aug/'+classLabel+'/'+X_val[eachIndex], 
                    datasetFolderName+'/validation/'+classLabel+'/'+X_val[eachIndex])
        
    train_datagen = ImageDataGenerator(
                      rescale=1./255,
                      zoom_range=0.20,
                      fill_mode="nearest")
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
        
    #Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)   
   
    # fit model
    history= model.fit(train_generator, 
                        epochs=epoch)
    
    predictions = Coromodel.predict_generator(validation_generator, verbose=1)
    yPredictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    # evaluate validation performance
    print("***Performance on Validation data***")    
    valAcc, valPrec, valFScore = my_metrics(true_classes, yPredictions)

Results for fold 1
Found 15938 images belonging to 5 classes.
Found 3985 images belonging to 5 classes.
Epoch 1/20
 87/125 [===================>..........] - ETA: 1:05:58 - loss: 0.6260 - accuracy: 0.7788

# Test phase

In [12]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False) 
predictions = model.predict(test_generator, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

testAcc,testPrec, testFScore = my_metrics(true_classes, yPredictions)
model.save(MODEL_FILENAME)

==============TEST RESULTS============
Found 45 images belonging to 3 classes.
1/1 [==============================] - 0s 452ms/step
Accuracy  : 0.8666666666666667
Precision : 0.8708333333333333
f1Score : 0.8680756395995552
[[12  0  3]
 [ 1 14  0]
 [ 2  0 13]]
